In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
pip install llama-index-utils-workflow

In [3]:
from llama_index.core.workflow import Workflow, step, StartEvent, StopEvent

class MyWorkflow(Workflow):
    
    @step
    async def my_step(self, event: StartEvent) -> StopEvent:
        return StopEvent(result="Hello World!")
    

workflow = MyWorkflow(timeout=10, verbose=True)

result = await workflow.run()
result

'Hello World!'

In [4]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

basic_workflow.html


In [5]:
from llama_index.core.workflow import Event

class FirstEvent(Event):
    first_output: str
    
class SecondEvent(Event):
    second_output: str

In [7]:
class MyWorkflow(Workflow):
    
    @step
    async def step_one(self, event: StartEvent) -> FirstEvent:
        print(event.first_input)
        return FirstEvent(first_output="First step complete")
    
    @step
    async def step_two(self, event: FirstEvent) -> SecondEvent:
        print(event.first_output)
        return SecondEvent(second_output="Second step complete")
    
    @step
    async def step_three(self, event: SecondEvent) -> StopEvent:
        print(event.second_output)
        return StopEvent(result="Workflow complete")
    

w = MyWorkflow(timeout=10, verbose=True)
result = await w.run(first_input="start the workflow")
print(result)

draw_all_possible_flows(MyWorkflow, filename="basic_workflow.html")

start the workflow
First step complete
Second step complete
Workflow complete
basic_workflow.html


Branch Workflow (IF ELSE)

In [8]:
class BranchA1Event(Event):
    payload: str
    
class BranchA2Event(Event):
    payload: str
    
class BranchB1Event(Event):
    payload: str
    
class BranchB2Event(Event):
    payload: str

In [10]:
import random

class BranchWorkflow(Workflow):
    
    @step
    async def start(self, event: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to Branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to Branch B")
            return BranchB1Event(payload="Branch B")
        
    @step
    async def step_a1(self, event: BranchA1Event) -> BranchA2Event:
        print(event.payload)
        return BranchA2Event(payload=event.payload)
    
    @step
    async def step_a2(self, event: BranchA2Event) -> StopEvent:
        print(event.payload)
        return StopEvent(payload="Branch A Complete")
    
    @step
    async def step_b1(self, event: BranchB1Event) -> BranchB2Event:
        print(event.payload)
        return BranchB2Event(payload=event.payload)
    
    @step
    async def step_b2(self, event: BranchB2Event) -> StopEvent:
        print(event.payload)
        return StopEvent(payload="Branch B Complete")
    
draw_all_possible_flows(BranchWorkflow, filename="branch_workflow.html")

branch_workflow.html


Loop Workflow

In [2]:
from llama_index.core.workflow import Event

class TextEvent(Event):
    text: str
    iteration: int = 0
    
class EvaluationEvent(Event):
    text: str
    feedback: str
    score: float
    iteration: int
    
class FinalTextEvent(Event):
    text: str
    iterations_required: int

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.core.workflow import Workflow, StartEvent, StopEvent, step
from llama_index.utils.workflow import draw_all_possible_flows

class TextRefinementWorkflow(Workflow):
    
    def __init__(self, timeout=60, verbose=False, max_iterations=5):
        super().__init__(timeout=timeout, verbose=verbose)
        self.llm = OpenAI(model="gpt-4o-mini")
        self.max_iterations = max_iterations
        
    
    @step
    async def initialize_text(self, event: StartEvent) -> TextEvent:
        initial_text = event.initial_text
        print(f"Initial Text Received (Iteration 0): {initial_text}")
        return TextEvent(text=initial_text, iteration=0)
    
    @step
    async def grade_text(self, event: TextEvent) -> EvaluationEvent | FinalTextEvent:
        if event.iteration >= self.max_iterations:
            print(f"Reached Maximum Iterations (Iteration {self.max_iterations})")
            return FinalTextEvent(text=event.text, iterations_required=event.iteration)
        
        prompt = f"""
        Evaluate the following text for clarity, coherence and conciseness
        
        Text: 
        {event.text}
        
        Provide:
        1. A score from 0-10(where 0 is least score and 10 is highest score)
        2. Specific feedback on how to improve
        
        Format your response exactly as:
        SCORE: [number]
        FEEDBACK: [your feedback]
        """
        
        response = await self.llm.acomplete(prompt)
        result = response.text.strip()
        
        score_line = [line for line in result.split("\n") if line.startswith("SCORE:")][0]
        score = float(score_line.split("SCORE:")[1].strip())
        
        feedback_line = [line for line in result.split("\n") if line.startswith("FEEDBACK:")][0]
        feedback = feedback_line.split("FEEDBACK:")[1].strip()
        
        print(f"Evaluation Iteration (Iteration {event.iteration})")
        print(f"Score: {score}")
        print(f"Feedback: {feedback}")
        
        if score > 8:
            print(f"Quality threshold met after {event.iteration} iterations!")
            return FinalTextEvent(text=feedback, iterations_required=event.iteration)
        
        return EvaluationEvent(text=event.text, feedback=feedback,
                               score=score, iteration=event.iteration)
        
    
    @step
    async def refine_text(self, event: EvaluationEvent) -> TextEvent:
        prompt = f"""
        Please improve the following text based on this feedback.
        
        Original Text:
        {event.text}
        
        Feedback:
        {event.feedback}
        
        Current Score: {event.score}
        
        Provide an improved version that addresses all the feeback points.
        Only return the improved text with no additional commentary
        """
        
        response = await self.llm.acomplete(prompt)
        result = response.text.strip()
        iteration = event.iteration + 1
    
        print(f"Refined Text (Iteration {iteration}: {result})")    
        return TextEvent(text=result, iteration=iteration)
    
    
    @step
    async def finalize_text(self, event: FinalTextEvent) -> StopEvent:
        result = {
            "text": event.text,
            "iterations": event.iterations_required
        }
        
        return StopEvent(result=result)
    
    

draw_all_possible_flows(TextRefinementWorkflow, filename="Text_refinement_workflow.html")        
    

Text_refinement_workflow.html


In [5]:
workflow = TextRefinementWorkflow()

initial_text = """
    Machine Learning is a process where computer systems can learn from data without being explicitly programmed.
    It uses algorithms to identify patterns in data and make predictions. Its used in many applications today.
"""

result = await workflow.run(initial_text=initial_text)

Initial Text Received (Iteration 0): 
    Machine Learning is a process where computer systems can learn from data without being explicitly programmed.
    It uses algorithms to identify patterns in data and make predictions. Its used in many applications today.

Evaluation Iteration (Iteration 0)
Score: 7.0
Feedback: The text is generally clear and conveys the main idea of machine learning effectively. However, it could be improved in terms of coherence and conciseness. Here are some suggestions:
Refined Text (Iteration 1: Machine learning is a process that enables computer systems to learn from data without explicit programming. By utilizing algorithms, it identifies patterns and makes predictions. Today, it is applied across various fields and industries.)
Evaluation Iteration (Iteration 1)
Score: 8.0
Feedback: The text is clear and coherent, effectively conveying the concept of machine learning and its applications. However, it could be more concise by eliminating some redundancy. 

In [6]:
result

{'text': 'Machine learning is a process that enables computer systems to learn from data without explicit programming. By utilizing algorithms, it identifies patterns and makes predictions. For example, in healthcare, machine learning can analyze medical images to detect diseases like cancer at an early stage. It is now applied in various fields, including finance and marketing.',
 'iterations': 5}

Maintaining Context

In [13]:
from llama_index.core.workflow import Context

class SetupEvent(Event):
    query: str
    
class QueryEvent(Event):
    query: str
    

class StatefulWorkFlow(Workflow):
    
    @step
    async def init(self, ctx: Context, event: StartEvent) -> SetupEvent | QueryEvent:
        db_config = await ctx.store.get("database_config", default=None)
        if db_config is None:
            print("Load DB Config")
            return SetupEvent(query=event.query)
        
        return QueryEvent(query=event.query)
    
    
    @step
    async def setup(self, ctx: Context, event:SetupEvent) ->StartEvent:
        await ctx.store.set("database_config", {"url": "jdbc://mock-url"})
        return StartEvent(query=event.query)
    
    @step
    async def query(self, ctx: Context, event: QueryEvent) -> StopEvent:
        print("Querying DB")
        return StopEvent(result = await ctx.store.get("database_config"))


workflow = StatefulWorkFlow()
result = await workflow.run(query="Load Customer Data")
result


Load DB Config
Querying DB


{'url': 'jdbc://mock-url'}